In [1]:
import asyncio
import logging
from concurrent.futures import ThreadPoolExecutor
import os
from pathlib import Path
from typing import Callable, List, Dict, Any
import asyncio
import logging
from concurrent.futures import ThreadPoolExecutor
import os
from pathlib import Path
from typing import List, Dict, Any, Callable
from malwaredoc.data.base import Base, Dictable
from pydantic import validator
import pickle
from typing import Optional, Union, Any
from malwaredoc.extract.pipeline import OfficePipeline, FilePipeline, FeatureExtractionPipeline, WordFilePipeline
from malwaredoc.extract.metadata import get_file_type, ExifInfo

from malwaredoc.extract.pipeline import FeatureExtractionPipeline

from malwaredoc.extract.util import convert_string_to_attribute_name
from pathlib import Path

In [2]:
logging.basicConfig(filename='feature_extraction_errors.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

executor = ThreadPoolExecutor(max_workers=min(32, os.cpu_count() + 4 if os.cpu_count() else 4))

async def run_async_task(func, *args, **kwargs) -> Any:
    """
    Execute any function asynchronously in a ThreadPoolExecutor.
    
    Args:
        func: The function to execute.
        *args: Positional arguments for the function.
        **kwargs: Keyword arguments for the function.
        
    Returns:
        The result of the function execution.
    """
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(executor, func, *args, **kwargs)

async def process_file_async(file_path: Path) -> Dict[str, Any]:
    """
    Asynchronously process a file using a dummy processing function.
    
    Args:
        file_path: Path to the file to be processed.
        
    Returns:
        A dictionary with the result of the processing.
    """
    try:
        result = await run_async_task(FeatureExtractionPipeline().process_file, file_path)
        return {"file_path": str(file_path), "result": result}
    except Exception as e:
        print(e)
        return {"file_path": str(file_path), "result": None}


async def process_files_async(file_paths: List[Path]) -> List[Dict[str, Any]]:
    """
    Asynchronously process a list of files.
    
    Args:
        file_paths: List of file paths to process.
        
    Returns:
        List of dictionaries with processing results.
    """
    tasks = [process_file_async(file_path) for file_path in file_paths]
    results = await asyncio.gather(*tasks)
    return results


In [3]:
dataset_directory = os.path.join(os.path.dirname(os.getcwd()), "malware_samples")
files_list = [Path(os.path.join(dataset_directory, os.path.join(file_name, file_name))) for file_name in os.listdir(dataset_directory)]

In [4]:
# import datetime
# start_time = datetime.datetime.now()
# results = await process_files_async(files_list)
# end_time = datetime.datetime.now()
# duration = end_time - start_time
# print(f"Execution time: {duration}")
